# RAG Workshop - Introduksjon

Velkommen til workshop om **Retrieval-Augmented Generation (RAG)** med Google Cloud Platform!

I denne workshopen skal vi bygge et komplett RAG-system som bruker:
- 📄 **Dokumentavgift 2025** som kunnskapsbase
- 🗄️ **Cloud SQL (PostgreSQL + pgvector)** som vector database
- 🤖 **Vertex AI** for embeddings og language models
- 🔐 **Secret Manager** for sikker håndtering av credentials

## Hva er RAG?

RAG kombinerer søk (retrieval) med generering (generation) for å gi LLM-er tilgang til ekstern kunnskap:

1. **Retrieval**: Finn relevant informasjon fra en kunnskapsbase
2. **Augmentation**: Berik spørsmålet med den hentede informasjonen  
3. **Generation**: Bruk en LLM til å generere svar basert på konteksten

### Hvorfor RAG?

- ✅ **Oppdatert informasjon**: Ikke begrenset til treningsdata
- ✅ **Faktabaserte svar**: Reduserer hallusinasjoner
- ✅ **Kilder**: Kan referere til hvor informasjonen kommer fra
- ✅ **Privat data**: Kan bruke bedriftsintern dokumentasjon

## Workshop-arkitektur

```
┌─────────────────┐
│ Dokumentavgift  │
│   PDF (2025)    │
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│ PDF Extraction  │ ← pymupdf4llm
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│    Chunking     │ ← LangChain
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│   Vertex AI     │ ← text-embedding-004
│   Embeddings    │
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│   Cloud SQL     │ ← PostgreSQL + pgvector
│  (pgvector)     │
└────────┬────────┘
         │
  Query │
         ▼
┌─────────────────┐
│ Vector Search   │ ← Similarity search
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│   Vertex AI     │ ← gemini-2.0-flash-001
│   Generation    │
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│     Answer      │
└─────────────────┘
```

## 1. Setup og konfigurasjon

### 🔧 Miljø-deteksjon

Denne notebooken fungerer både lokalt og i Google Colab!

In [ ]:
# Detekter om vi kjører i Colab
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🌐 Kjører i Google Colab")
else:
    print("💻 Kjører lokalt")

### ☁️ Google Colab Setup

**Hvis du bruker Google Colab, følg disse stegene:**

1. **Last opp PDF til Colab**
2. **Autentiser med GCP** (via Google Account)
3. **Installer pakker** (kjøres automatisk)

**Hvis du kjører lokalt, hopp over disse cellene!**

In [ ]:
# COLAB: Installer pakker
if IN_COLAB:
    print("📦 Installerer pakker for Colab...")
    !pip install -q pymupdf4llm langchain-text-splitters google-cloud-secret-manager google-genai psycopg2-binary pgvector
    print("✅ Pakker installert!")
else:
    print("⏭️  Hopper over - kjører lokalt")

In [ ]:
# COLAB: Last opp PDF
if IN_COLAB:
    from google.colab import files
    import shutil
    
    print("📤 Last opp dokumentavgift-2025.pdf:")
    uploaded = files.upload()
    
    # Flytt til forventet sti
    if uploaded:
        filename = list(uploaded.keys())[0]
        shutil.move(filename, 'dokumentavgift-2025.pdf')
        print(f"✅ PDF lastet opp: dokumentavgift-2025.pdf")
else:
    print("⏭️  Hopper over - kjører lokalt")

In [ ]:
# COLAB: GCP Autentisering
if IN_COLAB:
    from google.colab import auth
    print("🔐 Autentiserer med GCP...")
    auth.authenticate_user()
    print("✅ Autentisert!")
else:
    print("⏭️  Hopper over - kjører lokalt")

### 📦 Installer pakker (kun hvis nødvendig)

**Lokal**: Bruk `pip install -r requirements.txt` i terminalen  
**Colab**: Allerede installert i tidligere celle!

In [ ]:
# Installer pakker manuelt (kun hvis ikke installert via requirements.txt eller Colab)
if not IN_COLAB:
    print("💡 Tips: Installer fra requirements.txt i terminalen:")
    print("   pip install -r requirements.txt")
    print("\n⏭️  Hopper over automatisk installasjon lokalt")
else:
    print("✅ Pakker allerede installert i Colab")


💡 Tips: Installer heller fra requirements.txt i terminalen:
   pip install -r requirements.txt


In [79]:
# Import biblioteker
import os
import json
from pathlib import Path
from typing import List, Dict, Tuple

# PDF og teksthåndtering
import pymupdf4llm
from langchain_text_splitters import RecursiveCharacterTextSplitter

# GCP
from google.cloud import secretmanager
from google import genai

# Database
import psycopg2
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector

print("✅ Alle biblioteker importert!")

✅ Alle biblioteker importert!


### Konfigurer GCP-tilkobling

Vi setter opp tilkobling til GCP-prosjektet vårt og henter database-passordet fra Secret Manager.

In [80]:
# GCP konfigurasjon
PROJECT_ID = "data-science-faggruppe-rag"
REGION = "europe-west1"
SECRET_NAME = "postgres-password"

# Cloud SQL konfigurasjon (fra terraform)
DB_INSTANCE_NAME = "vector-db-instance"
DB_NAME = "vector_db"
DB_USER = "postgres"

print(f"�� Prosjekt: {PROJECT_ID}")
print(f"📍 Region: {REGION}")
print(f"📍 Database: {DB_NAME}")

�� Prosjekt: data-science-faggruppe-rag
📍 Region: europe-west1
📍 Database: vector_db


In [81]:
# Hent database-passord fra Secret Manager
def get_secret(project_id: str, secret_id: str, version_id: str = "latest") -> str:
    """
    Hent secret fra Google Secret Manager.
    
    Args:
        project_id: GCP prosjekt ID
        secret_id: Secret navn
        version_id: Versjon (default: latest)
    
    Returns:
        Secret verdi som string
    """
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("UTF-8")

# Hent passord
DB_PASSWORD = get_secret(PROJECT_ID, SECRET_NAME)
print("✅ Database-passord hentet fra Secret Manager")

✅ Database-passord hentet fra Secret Manager


**💡 Sikkerhetsmerknad:** Vi bruker Secret Manager for å unngå å hardkode passord i koden. Dette er best practice for produksjonssystemer!

In [82]:
# Hent Cloud SQL IP automatisk
import subprocess

try:
    # Prøv å hente IP fra gcloud
    result = subprocess.run(
        ['gcloud', 'sql', 'instances', 'describe', DB_INSTANCE_NAME,
         '--format=value(ipAddresses[0].ipAddress)'],
        capture_output=True,
        text=True,
        timeout=10
    )
    
    if result.returncode == 0:
        DB_HOST = result.stdout.strip()
        print(f"✅ Cloud SQL IP hentet automatisk: {DB_HOST}")
    else:
        # Fallback: Manuell input
        print("⚠️  Kunne ikke hente IP automatisk")
        print(f"   Error: {result.stderr}")
        DB_HOST = input("Vennligst skriv inn Cloud SQL IP: ")
        
except FileNotFoundError:
    print("⚠️  gcloud CLI ikke funnet")
    print("💡 Alternativ 1: Installer gcloud CLI")
    print("💡 Alternativ 2: Skriv inn IP manuelt")
    DB_HOST = input("Cloud SQL IP: ")
    
except subprocess.TimeoutExpired:
    print("⚠️  Timeout ved henting av IP")
    DB_HOST = input("Cloud SQL IP: ")

print(f"\n🔗 Database host: {DB_HOST}")

# Alternativ: Cloud SQL Proxy (hvis du bruker det)
# DB_HOST = "127.0.0.1"

✅ Cloud SQL IP hentet automatisk: 35.205.154.230

🔗 Database host: 35.205.154.230


## 2. Last inn og forbered dokument

Vi bruker **Dokumentavgift 2025** som kunnskapsbase. Dette er et enklere dokument enn statsbudsjettet, perfekt for å komme i gang!

In [83]:
# Last inn PDF og konverter til Markdown
if IN_COLAB:
    pdf_path = "dokumentavgift-2025.pdf"  # Opplastet fil i Colab
else:
    pdf_path = "../../data/dokumentavgift-2025.pdf"  # Lokal sti

print("📄 Ekstraherer tekst fra PDF...")
document_text = pymupdf4llm.to_markdown(pdf_path)

print(f"\n✅ PDF ekstrahert!")
print(f"📊 Statistikk:")
print(f"   - Tegn: {len(document_text):,}")
print(f"   - Linjer: {len(document_text.splitlines()):,}")
print(f"   - Ord: {len(document_text.split()):,}")

# Vis et utdrag
print("\n" + "="*80)
print("Utdrag fra dokumentet:")
print("="*80)
print(document_text[:500] + "...")

📄 Ekstraherer tekst fra PDF...

✅ PDF ekstrahert!
📊 Statistikk:
   - Tegn: 66,598
   - Linjer: 1,109
   - Ord: 8,503

Utdrag fra dokumentet:
## **ÅRSRUNDSKRIV FOR**
# **DOKUMENTAVGIFT** **2025**

1. januar 2025

#### **Skattedirektoratet**

Juridisk avdeling
Postboks 9200 Grønland

0134 OSLO
www.skatteetaten.no


### **Innhold**

1 Innledning .......................................................................................................................................... 4


1.1 Om Skattedirektoratets årsrundskriv ........................................................................................ 4


1.2 Hva er dokumenta...

✅ PDF ekstrahert!
📊 Statistikk:
   - Tegn: 66,598
   - Linjer: 1,109
   - Ord: 8,503

Utdrag fra dokumentet:
## **ÅRSRUNDSKRIV FOR**
# **DOKUMENTAVGIFT** **2025**

1. januar 2025

#### **Skattedirektoratet**

Juridisk avdeling
Postboks 9200 Grønland

0134 OSLO
www.skatteetaten.no


### **Innhold**

1 Innledning .........................................

### Chunking - Del opp dokumentet

Vi deler dokumentet i mindre chunks som passer for embedding og retrieval.

**Hvorfor chunking?**
- 📏 **Token-begrensninger**: Embedding-modeller har maks input-lengde
- 🎯 **Presisjon**: Mindre chunks gir mer presise søkeresultater
- 💰 **Kostnad**: Mindre context til LLM = lavere kostnader

Vi bruker LangChains `CharacterTextSplitter` som deler på dobbel-newline (paragrafer).

In [85]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],        # Del på dobbel-newline (paragraf)
    chunk_size=1000,         # Stor nok for hele avsnitt
    chunk_overlap=0,       # Overlap for kontekst
)

# Del opp dokumentet
chunks = text_splitter.create_documents([document_text])

print(f"✅ Dokumentet delt i {len(chunks)} chunks")
print(f"\n📊 Chunk-statistikk:")
chunk_lengths = [len(chunk.page_content) for chunk in chunks]
print(f"   - Gjennomsnitt: {sum(chunk_lengths)/len(chunk_lengths):.0f} tegn")
print(f"   - Min: {min(chunk_lengths)} tegn")
print(f"   - Max: {max(chunk_lengths)} tegn")

# Vis chunks som inneholder "dokumentavgift"
print(f"\n🔍 Chunks som inneholder 'hva er dokumentavgift':")
relevant_chunks = [
    (idx, chunk) for idx, chunk in enumerate(chunks) 
    if 'hva er dokumentavgift' in chunk.page_content.lower()
]

for idx, chunk in relevant_chunks[:3]:  # Vis maks 3
    print(f"\n{'='*80}")
    print(f"CHUNK {idx} ({len(chunk.page_content)} tegn):")
    print('='*80)
    print(chunk.page_content[:800])  # Vis første 800 tegn
    if len(chunk.page_content) > 800:
        print("\n... (resten kuttet for lesbarhet)")
    print('='*80)

if not relevant_chunks:
    print("  ⚠️  Ingen chunks funnet med 'hva er dokumentavgift'")
    print("\n📄 Viser i stedet første 2 chunks:")
    for idx in range(min(2, len(chunks))):
        print(f"\nCHUNK {idx}: {chunks[idx].page_content[:400]}...")


✅ Dokumentet delt i 84 chunks

📊 Chunk-statistikk:
   - Gjennomsnitt: 791 tegn
   - Min: 141 tegn
   - Max: 998 tegn

🔍 Chunks som inneholder 'hva er dokumentavgift':

CHUNK 0 (914 tegn):
## **ÅRSRUNDSKRIV FOR**
# **DOKUMENTAVGIFT** **2025**

1. januar 2025

#### **Skattedirektoratet**

Juridisk avdeling
Postboks 9200 Grønland

0134 OSLO
www.skatteetaten.no


### **Innhold**

1 Innledning .......................................................................................................................................... 4


1.1 Om Skattedirektoratets årsrundskriv ........................................................................................ 4


1.2 Hva er dokumentavgift? ........................................................................................................... 4


1.3 Hvilket regelverk gjelder? ......................................................................................................... 4


2 Avgiftsplikt .........................

... (reste

## 3. Vertex AI - Embeddings og LLM

Vi bruker Google Vertex AI for:
- **Embeddings**: `text-embedding-004` (768 dimensjoner)
- **Generation**: `gemini-2.0-flash-001` (rask og kostnadseffektiv)

In [86]:
# Konfigurer Vertex AI
client = genai.Client(vertexai=True, project=PROJECT_ID, location=REGION)

print("✅ Vertex AI konfigurert!")
print(f"   Project: {PROJECT_ID}")
print(f"   Region: {REGION}")
print(f"   Models: text-multilingual-embedding-002, gemini-2.0-flash-001")

✅ Vertex AI konfigurert!
   Project: data-science-faggruppe-rag
   Region: europe-west1
   Models: text-multilingual-embedding-002, gemini-2.0-flash-001


In [87]:
# Funksjon for å generere embeddings
def get_embedding(text: str, task_type: str = 'RETRIEVAL_DOCUMENT') -> List[float]:
    """
    Generer embedding for tekst ved hjelp av Vertex AI.
    
    Args:
        text: Tekst å embedde
        task_type: 'RETRIEVAL_DOCUMENT' for dokumenter, 'RETRIEVAL_QUERY' for søk
    
    Returns:
        Liste med floats (embedding-vektor)
    """
    client = genai.Client(vertexai=True, project=PROJECT_ID, location=REGION)
    response = client.models.embed_content(
        model='text-multilingual-embedding-002',
        contents=text,
        config={'task_type': task_type}
    )
    return response.embeddings[0].values

# Test embedding-generering
test_text = "Dette er en test av embedding-funksjonen."
test_embedding = get_embedding(test_text)

print(f"✅ Embedding generert!")
print(f"   - Dimensjoner: {len(test_embedding)}")
print(f"   - Type: {type(test_embedding[0])}")
print(f"   - Første 5 verdier: {test_embedding[:5]}")


✅ Embedding generert!
   - Dimensjoner: 768
   - Type: <class 'float'>
   - Første 5 verdier: [0.009539585560560226, -0.028674542903900146, -0.03958906978368759, 0.03002873621881008, 0.021501852199435234]


## 4. Cloud SQL - Vector Database

Vi kobler til Cloud SQL PostgreSQL med pgvector-extension for å lagre og søke i embeddings.

**pgvector** er en PostgreSQL-extension som gir:
- ⚡ Rask similarity search
- 📊 Støtte for forskjellige distance metrics (L2, cosine, inner product)
- 🔍 HNSW og IVFFlat indekser for skalerbarhet

In [88]:
# Koble til Cloud SQL
def get_db_connection():
    """
    Opprett tilkobling til Cloud SQL PostgreSQL.
    """
    conn = psycopg2.connect(
        host=DB_HOST,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        port=5432
    )
    return conn

# Test tilkobling
try:
    conn = get_db_connection()
    register_vector(conn)  # Registrer pgvector-typer
    
    cur = conn.cursor()
    cur.execute("SELECT version();")
    version = cur.fetchone()
    
    print("✅ Tilkoblet Cloud SQL!")
    print(f"   PostgreSQL versjon: {version[0][:50]}...")
    
    cur.close()
    conn.close()
    
except Exception as e:
    print(f"❌ Feil ved tilkobling: {e}")
    print("\n💡 Sjekk at:")
    print("   1. DB_HOST er satt til riktig IP")
    print("   2. Cloud SQL instance er oppe")
    print("   3. Firewall-regler tillater tilkobling")

✅ Tilkoblet Cloud SQL!
   PostgreSQL versjon: PostgreSQL 15.15 on x86_64-pc-linux-gnu, compiled ...


In [89]:
# Opprett tabell for chunks med embeddings
def create_chunks_table():
    """
    Opprett tabell for å lagre document chunks med embeddings.
    """
    conn = get_db_connection()
    register_vector(conn)
    cur = conn.cursor()
    
    # Drop eksisterende tabell (for workshop-formål)
    cur.execute("DROP TABLE IF EXISTS document_chunks;")
    
    # Opprett ny tabell
    cur.execute("""
        CREATE TABLE document_chunks (
            id SERIAL PRIMARY KEY,
            content TEXT NOT NULL,
            embedding vector(768),  -- text-embedding-004 har 768 dimensjoner
            metadata JSONB,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
    """)
    
    # Opprett index for raskere similarity search
    cur.execute("""
        CREATE INDEX ON document_chunks 
        USING ivfflat (embedding vector_cosine_ops)
        WITH (lists = 100);
    """)
    
    conn.commit()
    cur.close()
    conn.close()
    
    print("✅ Tabell 'document_chunks' opprettet med pgvector index!")

# Opprett tabell
create_chunks_table()

✅ Tabell 'document_chunks' opprettet med pgvector index!


### Generer embeddings og lagre i database

Nå genererer vi embeddings for alle chunks og lagrer dem i Cloud SQL. Dette kan ta noen minutter...

In [90]:
# Generer embeddings og lagre i database
def insert_chunks_with_embeddings(chunks: List, batch_size: int = 100):
    """
    Generer embeddings for chunks og lagre i database.
    
    Args:
        chunks: Liste med LangChain Document objekter
        batch_size: Antall chunks å prosessere samtidig
    """
    conn = get_db_connection()
    register_vector(conn)
    cur = conn.cursor()
    
    total = len(chunks)
    print(f"🔄 Prosesserer {total} chunks...\n")
    
    for i in range(0, total, batch_size):
        batch = chunks[i:i+batch_size]
        
        # Generer embeddings for batch
        for idx, chunk in enumerate(batch):
            global_idx = i + idx
            
            # Generer embedding
            embedding = get_embedding(chunk.page_content)
            
            # Metadata
            metadata = json.dumps({
                "chunk_index": global_idx,
                "length": len(chunk.page_content)
            })
            
            # Insert i database
            cur.execute(
                "INSERT INTO document_chunks (content, embedding, metadata) VALUES (%s, %s, %s)",
                (chunk.page_content, embedding, metadata)
            )
            
            if (global_idx + 1) % 10 == 0:
                print(f"   ✓ Prosessert {global_idx + 1}/{total} chunks")
        
        conn.commit()
    
    cur.close()
    conn.close()
    
    print(f"\n✅ Alle {total} chunks lagret i database!")

# Lagre chunks
insert_chunks_with_embeddings(chunks)

🔄 Prosesserer 84 chunks...

   ✓ Prosessert 10/84 chunks
   ✓ Prosessert 10/84 chunks
   ✓ Prosessert 20/84 chunks
   ✓ Prosessert 20/84 chunks
   ✓ Prosessert 30/84 chunks
   ✓ Prosessert 30/84 chunks
   ✓ Prosessert 40/84 chunks
   ✓ Prosessert 40/84 chunks
   ✓ Prosessert 50/84 chunks
   ✓ Prosessert 50/84 chunks
   ✓ Prosessert 60/84 chunks
   ✓ Prosessert 60/84 chunks
   ✓ Prosessert 70/84 chunks
   ✓ Prosessert 70/84 chunks
   ✓ Prosessert 80/84 chunks
   ✓ Prosessert 80/84 chunks

✅ Alle 84 chunks lagret i database!

✅ Alle 84 chunks lagret i database!


In [91]:
# Verifiser data i database
conn = get_db_connection()
cur = conn.cursor()

cur.execute("SELECT COUNT(*) FROM document_chunks;")
count = cur.fetchone()[0]

cur.execute("SELECT content, metadata FROM document_chunks LIMIT 3;")
samples = cur.fetchall()

print(f"📊 Database-statistikk:")
print(f"   - Antall chunks: {count}")
print(f"\n📄 Eksempel på lagrede chunks:\n")

for i, (content, metadata) in enumerate(samples, 1):
    print(f"Chunk {i}:")
    print(f"   Metadata: {metadata}")
    print(f"   Content: {content[:100]}...")
    print()

cur.close()
conn.close()

📊 Database-statistikk:
   - Antall chunks: 84

📄 Eksempel på lagrede chunks:

Chunk 1:
   Metadata: {'length': 914, 'chunk_index': 0}
   Content: ## **ÅRSRUNDSKRIV FOR**
# **DOKUMENTAVGIFT** **2025**

1. januar 2025

#### **Skattedirektoratet**

...

Chunk 2:
   Metadata: {'length': 969, 'chunk_index': 1}
   Content: 2.1 Avgiftspliktens omfang ............................................................................

Chunk 3:
   Metadata: {'length': 969, 'chunk_index': 2}
   Content: 3.4 Registerførerens fastsettelse av avgiftsgrunnlaget – "åpenbart for lavt" ..........................



## 5. Retrieval - Søk etter relevante chunks

Nå kan vi søke i vector database med semantic search!

**Slik fungerer det:**
1. Konverter brukerens spørsmål til embedding
2. Søk etter chunks med mest lignende embeddings (cosine similarity)
3. Returner topp-k mest relevante chunks

In [92]:
# Søkefunksjon
def search_similar_chunks(query: str, top_k: int = 5) -> List[Dict]:
    """
    Søk etter lignende chunks basert på query.
    
    Args:
        query: Søkestreng
        top_k: Antall resultater å returnere
    
    Returns:
        Liste med relevante chunks og similarity scores
    """
    query_embedding = get_embedding(query, task_type='RETRIEVAL_QUERY')
    
    # Søk i database
    conn = get_db_connection()
    register_vector(conn)
    cur = conn.cursor()
    
    cur.execute(
        """
        SELECT 
            id,
            content,
            metadata,
            1 - (embedding <=> %s::vector) as similarity
        FROM document_chunks
        ORDER BY embedding <=> %s::vector
        LIMIT %s;
        """,
        (query_embedding, query_embedding, top_k)
    )
    
    results = []
    for row in cur.fetchall():
        results.append({
            'id': row[0],
            'content': row[1],
            'metadata': row[2],
            'similarity': float(row[3])
        })
    
    cur.close()
    conn.close()
    
    return results

# Test søk
test_query = "Hva er dokumentavgift?"
results = search_similar_chunks(test_query, top_k=3)

print(f"🔍 Søk: '{test_query}'\n")
print("="*80)

if not results:
    print("⚠️  Ingen resultater funnet! Sjekk at databasen inneholder data.")
else:
    for i, result in enumerate(results, 1):
        print(f"\nResultat {i} (Similarity: {result['similarity']:.4f}):")
        print("-"*80)
        print(f"Lengde: {len(result['content'])} tegn")
        print(f"Innhold: {result['content'][:500]}...")
        print("="*80)



🔍 Søk: 'Hva er dokumentavgift?'


Resultat 1 (Similarity: 0.7768):
--------------------------------------------------------------------------------
Lengde: 911 tegn
Innhold: _[(dal. § 7](https://lovdata.no/lov/1975-12-12-59/§7)_ _femte ledd annet punktum)_
Ved offentlig jordskifte skal dokumentavgiften beregnes som ved oppløsning av sameie (se punkt 4.4),
jf. dal. § 7 femte ledd annet punktum. Dette innebærer at avgiften beregnes av verdiforskyvningen
innenfor jordskifteområdet, sett under ett.

#### **4.16 Annet**

_[(helseforetaksloven § 50](https://lovdata.no/lov/2001-06-15-93/§50)_ _tredje ledd og_ _[inndelingsloven § 14)](https://lovdata.no/lov/2001-06-15-70/§1...

Resultat 2 (Similarity: 0.7157):
--------------------------------------------------------------------------------
Lengde: 980 tegn
Innhold: På denne bakgrunn refunderes betalt dokumentavgift når det foreligger rettskraftig dom eller rettsforlik
som kjenner dokumentbeskrevet rettsstiftelse ugyldig fra først av. Ugyldighe

## 6. Generation - Generer svar med Gemini

Siste steg: Kombiner retrieved chunks med brukerens spørsmål og bruk Gemini til å generere et svar.

In [93]:
# Genereringsfunksjon
def generate_answer(query: str, context_chunks: List[Dict]) -> str:
    """
    Generer svar basert på query og context.
    
    Args:
        query: Brukerens spørsmål
        context_chunks: Relevante chunks fra retrieval
    
    Returns:
        Generert svar
    """
    # Bygg context fra chunks
    context = "\n\n".join([
        f"[Kilde {i+1}]\n{chunk['content']}" 
        for i, chunk in enumerate(context_chunks)
    ])
    
    # Bygg prompt
    prompt = f"""Du er en hjelpsom assistent som svarer på spørsmål om dokumentavgift basert på norsk regelverk.

Svar på spørsmålet basert på konteksten nedenfor. Hvis konteksten ikke inneholder nok informasjon til å gi et komplett svar, gi et delvis svar basert på det som er tilgjengelig.

KONTEKST:
{context}

SPØRSMÅL: {query}

SVAR:"""
    
    # Generer svar med Gemini
    client = genai.Client(vertexai=True, project=PROJECT_ID, location=REGION)
    response = client.models.generate_content(
        model='gemini-2.5-flash',
        contents=prompt
    )
    
    return response.text

# Test generering
answer = generate_answer(test_query, results)

print(f"❓ Spørsmål: {test_query}\n")
print("="*80)
print("📝 Svar:")
print("="*80)
print(answer)


❓ Spørsmål: Hva er dokumentavgift?

📝 Svar:
Konteksten gir ikke en direkte definisjon av hva dokumentavgift er.

Imidlertid fremgår det at det er en avgift som:
*   Beregnes ved offentlig jordskifte, basert på verdiforskyvningen (Kilde 1).
*   Vanligvis skal betales for overføringer i grunnboken og tinglysing av overføringer, med visse unntak som for overføringer etter helseforetaksloven og grenseendringer etter inndelingsloven (Kilde 1).
*   Kan refunderes dersom en dokumentert rettsstiftelse kjennes ugyldig, eller der et eldre rettserverv går foran et yngre (Kilde 2).


## 7. Komplett RAG Pipeline

La oss pakke alt sammen i én funksjon!

In [94]:
# Komplett RAG pipeline
def rag_query(query: str, top_k: int = 5) -> Dict:
    """
    Komplett RAG pipeline: Retrieve + Generate.
    
    Args:
        query: Brukerens spørsmål
        top_k: Antall chunks å hente
    
    Returns:
        Dict med query, answer, og sources
    """
    print(f"🔍 Søker etter relevante chunks...")
    chunks = search_similar_chunks(query, top_k=top_k)
    
    print(f"✓ Fant {len(chunks)} relevante chunks")
    print(f"🤖 Genererer svar...")
    
    answer = generate_answer(query, chunks)
    
    print(f"✓ Svar generert!\n")
    
    return {
        'query': query,
        'answer': answer,
        'sources': chunks
    }

# Test med flere spørsmål
queries = [
    "Hva er dokumentavgift?",
    "Hvem må betale dokumentavgift?",
    "Hva er avgiftssatsen for dokumentavgift?"
]

for query in queries:
    print("\n" + "="*80)
    result = rag_query(query, top_k=3)
    print("="*80)
    print(f"\n❓ {result['query']}")
    print(f"\n📝 {result['answer']}")
    print(f"\n📚 Basert på {len(result['sources'])} kilder")


🔍 Søker etter relevante chunks...
✓ Fant 2 relevante chunks
🤖 Genererer svar...
✓ Fant 2 relevante chunks
🤖 Genererer svar...
✓ Svar generert!


❓ Hva er dokumentavgift?

📝 Konteksten gir ikke en direkte definisjon av hva dokumentavgift er.

Basert på informasjonen kan det likevel utledes at dokumentavgift er en avgift som:
*   **Beregningsgrunnlag:** I visse tilfeller, som ved offentlig jordskifte, beregnes den av verdiforskyvningen innenfor jordskifteområdet.
*   **Anvendelsesområde:** Den nevnes i forbindelse med overføringer i grunnboken og tinglysing av overføringer.
*   **Unntak:** Det skal ikke betales dokumentavgift for overføringer i grunnboken som foretas i medhold av helseforetaksloven, eller for tinglysing av overføringer som er en direkte følge av grenseendringer etter inndelingsloven.
*   **Refusjon:** Betalt dokumentavgift kan refunderes under spesielle omstendigheter, for eksempel når en rettsstiftelse kjennes ugyldig eller et eldre rettserverv går foran et yngre.

📚 B

## 🎉 Oppsummering

Gratulerer! Du har nå bygget et komplett RAG-system med:

### ✅ Hva vi har lært

1. **PDF-ekstraksjon** med pymupdf4llm
2. **Chunking** med LangChain
3. **Embeddings** med Vertex AI (text-embedding-004)
4. **Vector database** med Cloud SQL + pgvector
5. **Similarity search** med cosine distance
6. **Answer generation** med Gemini 1.5 Flash
7. **Secret management** med Google Secret Manager

### 🚀 Neste steg

I de neste delene av workshopen skal vi se på:
- **Avansert chunking**: Hierarkisk og semantisk chunking
- **Hybrid search**: Kombinere vector search med keyword search
- **Re-ranking**: Forbedre retrieved chunks
- **Evaluering**: Måle kvalitet på RAG-systemet

### 💡 Tips for produksjon

- 🔒 Bruk private IP for Cloud SQL
- 📊 Implementer logging og monitoring
- ⚡ Optimaliser med batch processing
- 🧪 Test med forskjellige embedding-modeller
- �� Skaler med større vector indexes (HNSW)

**Lykke til videre!** ��

---

## 💾 Checkpoint - Eksporter data for neste notebook

Før du går videre til neste del av workshopen, eksporter data slik at du kan fortsette der du slapp!

In [ ]:
# Eksporter checkpoint til JSON for neste notebook
checkpoint_data = {
    'project_id': PROJECT_ID,
    'region': REGION,
    'db_config': {
        'host': DB_HOST,
        'database': DB_NAME,
        'user': DB_USER,
        # Passord lastes fra Secret Manager i neste notebook
    },
    'document': {
        'name': 'Dokumentavgift 2025',
        'path': pdf_path,
        'total_chunks': len(chunks),
        'text_length': len(document_text)
    },
    'embeddings': {
        'model': 'text-multilingual-embedding-002',
        'dimensions': 768
    },
    'chunks_in_db': True
}

# Lagre checkpoint
checkpoint_file = 'workshop_checkpoint.json'
with open(checkpoint_file, 'w', encoding='utf-8') as f:
    json.dump(checkpoint_data, f, indent=2, ensure_ascii=False)

print(f"✅ Checkpoint lagret: {checkpoint_file}")
print(f"\n📊 Oppsummering:")
print(f"   - Chunks i database: {checkpoint_data['document']['total_chunks']}")
print(f"   - Embedding model: {checkpoint_data['embeddings']['model']}")
print(f"   - Database: {checkpoint_data['db_config']['database']}")
print(f"\n💡 Bruk denne filen til å starte neste notebook!")

### 🔜 Klar for neste steg?

Du har nå:
- ✅ Bygget en grunnleggende RAG pipeline
- ✅ Lagret chunks med embeddings i Cloud SQL
- ✅ Eksportert checkpoint-data

**Neste notebooks:**

1. **Chunking-strategier** (`chunking.ipynb`)
   - Sammenlign forskjellige chunking-metoder
   - Hierarkisk vs semantisk chunking
   - Forbedre retrieval-kvalitet

2. **Avansert søk** 
   - Hybrid search (vector + keyword)
   - Query rewriting
   - Metadata filtering

3. **Re-ranking og evaluering**
   - Cross-encoder re-ranking
   - RAG-spesifikke metrics
   - Kvalitetsmåling

**Start neste notebook med checkpoint-data:**
```python
import json
with open('workshop_checkpoint.json', 'r') as f:
    checkpoint = json.load(f)
```

## (Valgfri) Opprydding

Hvis du vil slette data fra databasen:

In [ ]:
# Slett alle chunks (kjør bare hvis du vil starte på nytt)
# conn = get_db_connection()
# cur = conn.cursor()
# cur.execute("DROP TABLE IF EXISTS document_chunks;")
# conn.commit()
# cur.close()
# conn.close()
# print("✅ Tabell slettet")